In [ ]:
# default_exp preprocessing

# Preprocessing data

> Inspecting any particular irregularities and general preparation of the data for modelling.

In [ ]:
#export
import pandas as pd
from pathlib import Path
import os
import plotly.graph_objects as go
import plotly.express as px
import numpy as np
import typing
import pickle

from sklearn import linear_model, tree, model_selection, ensemble

from fastai.tabular.all import *

In [ ]:
pd.options.plotting.backend = "plotly"

In [ ]:
base_path = Path("../data")

In [ ]:
csvs = sorted([base_path/v for v in os.listdir(base_path) if v.endswith('.csv')])
csvs

In [ ]:
train_csv = csvs[3]
train_weather_csv = csvs[-1]
test_csv = csvs[2]
test_weather_csv = csvs[-2]
meta_csv = csvs[0]

train_csv, train_weather_csv, test_csv, test_weather_csv, meta_csv

## Loading

In [ ]:
%%time
train = pd.read_csv(train_csv, parse_dates=['timestamp'])
train.head()

In [ ]:
%%time
test = pd.read_csv(test_csv, parse_dates=['timestamp'])
test.head()

In [ ]:
%%time
weather_train = pd.read_csv(train_weather_csv, parse_dates=['timestamp'])
weather_train.head()

In [ ]:
%%time
weather_test = pd.read_csv(test_weather_csv, parse_dates=['timestamp'])
weather_test.head()

In [ ]:
%%time
building = pd.read_csv(meta_csv)
building.head()

## Inspection of the data

In [ ]:
train.head()

In [ ]:
len(train)

Kicking out outlying measurements

In [ ]:
%%time
train_meter_stats = (train.groupby(['meter'])['meter_reading']
                     .describe(percentiles=[.05, .25, .5, .75, .95]))
train_meter_stats

In [ ]:
%%time
mask = pd.concat([(train['meter']==m) & (train['meter_reading'] < 1e2*grp['95%'].iloc[0])
         for m, grp in train_meter_stats.groupby(['meter'])], axis=1).any(axis=1)
mask

In [ ]:
mask.sum() / len(mask)

In [ ]:
train = train.loc[mask]

In [ ]:
(train_meter_stats['95%'] == 0 ).sum()

TODO: there are some building id and meter combinations with timeseries essentially only consisting of the value zero $\rightarrow$ how to clean up? 

TODO: find out if 0s are trailing, leading or randomly intermittent 

In [ ]:
%%time
share_of_zeros = lambda x: (x==0).sum()/len(x)
df_zeros = (train.groupby(['building_id', 'meter'])['meter_reading']
            .apply(share_of_zeros)
            .sort_values()
            .to_frame()
            .rename(columns={'meter_reading':'fraction 0s'}))
df_zeros

In [ ]:
df_zeros['ix'] = df_zeros.groupby(['meter']).cumcount()
df_zeros.head()

In [ ]:
px.scatter(df_zeros.reset_index(level=1), x='ix', y='fraction 0s', color='meter')

## Radically merging all the data

In [ ]:
#export
def radical_merging(df:pd.DataFrame, building:pd.DataFrame, 
                    weather:pd.DataFrame, n_sample:int=None,
                    training:bool=True):
    
    tmp = df.copy(deep=True)

    bid_col = 'building_id'
    sid_col = 'site_id'
    time_col = 'timestamp'
    target_col = 'meter_reading'
    
    categorical = ['meter', 'primary_use', 'cloud_coverage', bid_col, sid_col]
    continuous = ['square_feet', 'year_built', 'floor_count', 
                  'air_temperature', 'dew_temperature',
                  'precip_depth_1_hr', 'sea_level_pressure', 'wind_direction',
                  'wind_speed']

    x_cols = [bid_col, 'meter', target_col, time_col] if training \
            else [bid_col, 'meter', time_col]
    X = tmp.loc[:,x_cols].copy()

    X = pd.merge(X, building, on=bid_col, how='left')
    X = pd.merge(X, weather, on=[sid_col, time_col], how='left')

    #return_cols =  categorical + continuous + [target_col,]  # time_col

    #X = X.loc[:,return_cols]
    if n_sample is not None:
        X = X.sample(n_sample)
        
    if training:
        X[target_col] = np.log(X[target_col] + 1)
        
    X = add_datepart(X, time_col)
    categorical.extend(['timestampMonth', 'timestampWeek', 'timestampDay',
                        'timestampDayofweek', 'timestampDayofyear', 'timestampIs_month_end',
                        'timestampIs_month_start', 'timestampIs_quarter_end',
                        'timestampIs_quarter_start', 'timestampIs_year_end',
                        'timestampIs_year_start'])
    
    continuous.extend(['timestampYear', 'timestampElapsed'])
        
    X = X.loc[:, [col for col in X.columns.values if col not in [time_col]]]
    
    missing_cont = [col for col in continuous if col not in X.columns]
    missing_cat = [col for col in categorical if col not in X.columns]
    assert len(missing_cat) == 0, f'{missing_cat} not in X!'
    assert len(missing_cont) == 0, f'{missing_cont} not in X!'
    
    X.loc[:,continuous] = X.loc[:,continuous].astype(float)
    X.loc[:,categorical] = X.loc[:,categorical].astype('category')
    
    return X, continuous, categorical

Generating train / validate features

In [ ]:
%%time
n_sample = None  #10000
X, continuous, categorical = radical_merging(train.copy(), building, weather_train,
                    n_sample=n_sample)

In [ ]:
%%time
X.to_parquet('../data/X.parquet')

Storing variable types

In [ ]:
with open('../data/var_types.pckl', 'wb') as f:
    pickle.dump({'cont':continuous, 'cat':categorical}, f)

Generating test set features

In [ ]:
%%time
X_test, _, _ = radical_merging(test.copy(), building, weather_test,
                    n_sample=None, training=False)

In [ ]:
X_test.to_parquet('../data/X_test.parquet')